## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [27]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data.shape, data

((60, 4, 1000),
 array([[[ -3.1149735 ,   7.1671543 ,   1.4840266 , ...,  13.99208   ,
            1.0122937 ,   7.448365  ],
         [ -3.970488  ,   4.261105  ,  19.453253  , ...,  -3.7643213 ,
           -3.1151319 ,   6.489771  ],
         [ -1.2794845 ,  10.364174  , -25.136284  , ...,  -9.842807  ,
           17.9597    ,  12.99544   ],
         [-18.772863  ,  -3.4441104 ,   4.2517533 , ..., -15.298768  ,
          -19.565794  ,   6.030257  ]],
 
        [[ -9.4167795 ,  -4.7108417 ,   6.5704136 , ..., -16.155935  ,
          -17.585062  , -20.249805  ],
         [ -2.6972098 , -10.928801  ,  13.520526  , ...,   4.6124363 ,
            3.2898815 ,  -5.789578  ],
         [ -9.5732565 ,   7.6940613 ,  -1.3763493 , ...,   1.9816842 ,
            1.7398272 ,   2.6637778 ],
         [-12.282049  ,  21.207176  ,  13.850941  , ...,  11.342485  ,
            0.41475245,   1.109602  ]],
 
        [[ -2.9517212 ,  11.283921  ,   1.0081706 , ...,   7.887978  ,
            8.043406  ,   4

In [26]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[1] - segment) / step) + 1  # Adjusted for the second dimension
ids = np.arange(n_win) * step

# Janelas do dado no domínio do tempo
chunks_time = np.array([data[:, k:(k + segment), :].transpose(0, 2, 1) for k in ids])

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=segment, noverlap=step-1)  # Adjusted noverlap
chunks_freq = chunks_freq.transpose(0, 2, 3, 1)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Domínio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Domínio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (28, 1000, 4)


C:\Users\caio-\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 64 is greater than input length  = 4, using nperseg = 4
  warnings.warn('nperseg = {0:d} is greater than input length '


ValueError: noverlap must be less than nperseg.

*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).